# Set kernel R shell

In [2]:
.libPaths("/home/spuccio/miniconda3/envs/scrnseq/lib/R/library")
suppressMessages(suppressWarnings(require(SingleCellExperiment)))
suppressMessages(suppressWarnings(require(Seurat)))
suppressMessages(suppressWarnings(require(tidyverse)))
suppressMessages(suppressWarnings(require(Matrix)))
suppressMessages(suppressWarnings(require(scales)))
suppressMessages(suppressWarnings(require(cowplot)))
suppressMessages(suppressWarnings(require(RCurl)))
suppressMessages(suppressWarnings(require(DropletUtils)))
for (file in c("PTZ_6","PTZ_7","PTZ_8","PTZ_10","PTZ_11","PTZ_12","PTZ_14","PTZ_15","PTZ_16")){
        seurat_data <- Read10X(data.dir = paste0("/mnt/lugli/SP028_Autoimmunity/dataset/IBD_RNA_sequencing_result/IBD_counts/", file))
        seurat_obj <- CreateSeuratObject(counts = seurat_data, 
                                         min.cells = 3, min.features = 100, 
                                         project = file)
        assign(file, seurat_obj)
}

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”


In [4]:
merged_seurat_PTZ6_7 <- merge(x = PTZ_6, 
                       y = PTZ_7, 
                       add.cell.id = c("PTZ_6", "PTZ_7"))

In [39]:
merged_seurat_PTZ6_7

An object of class Seurat 
18019 features across 5645 samples within 1 assay 
Active assay: RNA (18019 features, 0 variable features)

In [5]:
merged_seurat_PTZ6_7_8 <- merge(x = merged_seurat_PTZ6_7, 
                       y = PTZ_8, 
                       add.cell.id = c("merged","PTZ_8"))

In [40]:
merged_seurat_PTZ6_7_8

An object of class Seurat 
18851 features across 10756 samples within 1 assay 
Active assay: RNA (18851 features, 0 variable features)

In [8]:
merged_seurat_PTZ6_7_8_10 <- merge(x = merged_seurat_PTZ6_7_8, 
                       y = PTZ_10, 
                       add.cell.id = c("merged","PTZ_10"))

An object of class Seurat 
15412 features across 1875 samples within 1 assay 
Active assay: RNA (15412 features, 0 variable features)

In [9]:
merged_seurat_PTZ6_7_8_10_11 <- merge(x = merged_seurat_PTZ6_7_8_10, 
                       y = PTZ_11, 
                       add.cell.id = c("merged","PTZ_11"))

In [10]:
merged_seurat_PTZ6_7_8_10_11_12 <- merge(x = merged_seurat_PTZ6_7_8_10_11, 
                       y = PTZ_12, 
                       add.cell.id = c("merged","PTZ_12"))

In [11]:
merged_seurat_PTZ6_7_8_10_11_12_14 <- merge(x = merged_seurat_PTZ6_7_8_10_11_12, 
                       y = PTZ_14, 
                       add.cell.id = c("merged","PTZ_14"))

In [12]:
merged_seurat_PTZ6_7_8_10_11_12_14_15 <- merge(x = merged_seurat_PTZ6_7_8_10_11_12_14, 
                       y = PTZ_15, 
                       add.cell.id = c("merged","PTZ_15"))

In [14]:
merged_seurat <- merge(x = merged_seurat_PTZ6_7_8_10_11_12_14_15, 
                       y = PTZ_16, 
                       add.cell.id = c("merged","PTZ_16"))

In [15]:
# Add number of genes per UMI for each cell to metadata
merged_seurat$log10GenesPerUMI <- log10(merged_seurat$nFeature_RNA) / log10(merged_seurat$nCount_RNA)

In [16]:
# Compute percent mito ratio
merged_seurat$mitoRatio <- PercentageFeatureSet(object = merged_seurat, pattern = "^MT-")
merged_seurat$mitoRatio <- merged_seurat@meta.data$mitoRatio / 100

In [17]:
# Create metadata dataframe
metadata <- merged_seurat@meta.data

In [18]:
merged_seurat@meta.data <- metadata

In [19]:
# Add cell IDs to metadata
metadata$cells <- rownames(metadata)

# Rename columns
metadata <- metadata %>%
        dplyr::rename(seq_folder = orig.ident,
                      nUMI = nCount_RNA,
                      nGene = nFeature_RNA)

In [31]:
row.names(metadata) <- gsub(x = row.names(metadata), pattern = "merged_", replacement = "")

In [34]:
row.names(metadata) <- gsub(x = row.names(metadata), pattern = "PTZ_[0-9]_", replacement = "")

In [36]:
row.names(metadata) <- gsub(x = row.names(metadata), pattern = "PTZ_[0-9][0-9]", replacement = "")

In [37]:
write.table(metadata,"/mnt/lugli/SP028_Autoimmunity/dataset/IBD_RNA_sequencing_result/IBD_counts/Merged_metadata.txt",sep="\t",quote=F,row.names = T)

In [45]:
write10xCounts(x = merged_seurat@assays$RNA@counts, path = "/mnt/lugli/SP028_Autoimmunity/dataset/IBD_RNA_sequencing_result/IBD_counts/10Xmerge",version = "2", overwrite = TRUE)